In [1]:
import os, glob, random
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import pandas as pd

In [2]:
# ===== BraTS 2020 LOAD =====
base_path_20 = r"C:\Users\dados\Downloads\archive\BraTS2020_TrainingData\MICCAI_BraTS2020_TrainingData"

patient_dirs_20 = [
    os.path.join(base_path_20, d)
    for d in os.listdir(base_path_20)
    if os.path.isdir(os.path.join(base_path_20, d)) and d.startswith("BraTS20_Training_")
]
assert len(patient_dirs_20) > 0, "Nenhuma pasta BraTS20_Training_* encontrada."

patient_folder_20 = random.choice(patient_dirs_20)
print("Paciente (2020) escolhido:", os.path.basename(patient_folder_20))

def find_modal_paths_20(patient_folder):
    patt = lambda suf: glob.glob(os.path.join(patient_folder, f"*_{suf}.nii"))
    paths = {
        "t1":    (patt("t1")    or [None])[0],
        "t1ce":  (patt("t1ce")  or [None])[0],
        "t2":    (patt("t2")    or [None])[0],
        "flair": (patt("flair") or [None])[0],
        "seg":   (patt("seg")   or [None])[0],
    }
    miss = [k for k,v in paths.items() if v is None]
    if miss:
        raise FileNotFoundError(f"Arquivos ausentes em {patient_folder}: {miss}")
    return paths

paths20 = find_modal_paths_20(patient_folder_20)

# Carrega volumes (2020)
t1_20    = nib.load(paths20["t1"]).get_fdata()
t1c_20   = nib.load(paths20["t1ce"]).get_fdata()
t2_20    = nib.load(paths20["t2"]).get_fdata()
flair_20 = nib.load(paths20["flair"]).get_fdata()
seg_20   = nib.load(paths20["seg"]).get_fdata()

mask_bin_20 = (seg_20 > 0).astype(np.uint8)
Z20 = t1_20.shape[2]

Paciente (2020) escolhido: BraTS20_Training_109


In [3]:
# ===== BraTS 2020 plot =====
def viewer_20(z):
    # Cria a figura
    plt.figure(figsize=(20, 8))

    # ---------- LINHA DE CIMA (imagens + máscara original) ----------

    # (1) T1
    plt.subplot(2, 5, 1)
    plt.imshow(t1_20[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T1")
    plt.axis("off")

    # (2) T1c
    plt.subplot(2, 5, 2)
    plt.imshow(t1c_20[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T1c")
    plt.axis("off")

    # (3) T2w
    plt.subplot(2, 5, 3)
    plt.imshow(t2_20[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T2w")
    plt.axis("off")

    # (4) FLAIR
    plt.subplot(2, 5, 4)
    plt.imshow(flair_20[:, :, z].T, cmap="gray", origin="lower")
    plt.title("FLAIR")
    plt.axis("off")

    # (5) Máscara ORIGINAL (multi-classes)
    plt.subplot(2, 5, 5)
    plt.imshow(seg_20[:, :, z].T, cmap="nipy_spectral", origin="lower")
    plt.title("Máscara (orig)")
    plt.axis("off")

    # ---------- LINHA DE BAIXO (overlays + máscara binária) ----------

    # Prepara a máscara binária da slice (H, W)
    m = (mask_bin_20[:, :, z].T > 0).astype(np.uint8)

    # (6) T1 + máscara
    plt.subplot(2, 5, 6)
    plt.imshow(t1_20[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T1 + Másc")
    plt.axis("off")

    # (7) T1c + máscara
    plt.subplot(2, 5, 7)
    plt.imshow(t1c_20[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T1c + Másc")
    plt.axis("off")

    # (8) T2w + máscara
    plt.subplot(2, 5, 8)
    plt.imshow(t2_20[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T2w + Másc")
    plt.axis("off")

    # (9) FLAIR + máscara
    plt.subplot(2, 5, 9)
    plt.imshow(flair_20[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("FLAIR + Másc")
    plt.axis("off")

    # (10) Máscara BINÁRIA
    plt.subplot(2, 5, 10)
    plt.imshow(m, cmap="gray", origin="lower")
    plt.title("Máscara (bin)")
    plt.axis("off")

    # Ajusta espaçamento do grid
    plt.tight_layout()

    # Exibe a figura
    plt.show()

# Slider
interact(viewer_20, z=(0, Z20-1))


interactive(children=(IntSlider(value=77, description='z', max=154), Output()), _dom_classes=('widget-interact…

<function __main__.viewer_20(z)>

In [4]:
# -------- BraTS 2020 SHAPE --------
assert all(n in globals() for n in ["t1_20","t1c_20","t2_20","flair_20","seg_20"])

rows_2020 = [
    {"dataset":"BraTS2020", "modalidade":"t1",    "H": t1_20.shape[0],   "W": t1_20.shape[1],   "Z": t1_20.shape[2]},
    {"dataset":"BraTS2020", "modalidade":"t1ce",  "H": t1c_20.shape[0],  "W": t1c_20.shape[1],  "Z": t1c_20.shape[2]},
    {"dataset":"BraTS2020", "modalidade":"t2",    "H": t2_20.shape[0],   "W": t2_20.shape[1],   "Z": t2_20.shape[2]},
    {"dataset":"BraTS2020", "modalidade":"flair", "H": flair_20.shape[0],"W": flair_20.shape[1],"Z": flair_20.shape[2]},
    {"dataset":"BraTS2020", "modalidade":"seg",   "H": seg_20.shape[0],  "W": seg_20.shape[1],  "Z": seg_20.shape[2]},
]
df_2020 = pd.DataFrame(rows_2020).sort_values("modalidade").reset_index(drop=True)
df_2020

,dataset,modalidade,H,W,Z
0,BraTS2020,flair,240,240,155
1,BraTS2020,seg,240,240,155
2,BraTS2020,t1,240,240,155
3,BraTS2020,t1ce,240,240,155
4,BraTS2020,t2,240,240,155


In [5]:
# Usa base_path_20 previamente definido no notebook
assert 'base_path_20' in globals(), "Defina base_path_20 antes desta célula."

# ---------- Config (2020) ----------
_modalidades_20 = ["t1", "t1ce", "t2", "flair", "seg"]

# ---------- Helpers (2020) ----------
def _list_patient_dirs_20(base, prefix="BraTS20_Training_"):
    return [
        os.path.join(base, d)
        for d in os.listdir(base)
        if os.path.isdir(os.path.join(base, d)) and d.startswith(prefix)
    ]

def _find_modal_paths_20(patient_folder):
    # Busca recursiva; aceita .nii e .nii.gz (no 2020 costuma ser .nii)
    hits = {m: None for m in _modalidades_20}
    for root, _, files in os.walk(patient_folder):
        for f in files:
            fl = f.lower()
            if not (fl.endswith(".nii") or fl.endswith(".nii.gz")):
                continue
            p = os.path.join(root, f)
            if "_t1ce" in fl:
                hits["t1ce"] = hits["t1ce"] or p
            elif "_t1" in fl and "t1ce" not in fl:
                hits["t1"] = hits["t1"] or p
            elif "_t2" in fl and "flair" not in fl:
                hits["t2"] = hits["t2"] or p
            elif "_flair" in fl:
                hits["flair"] = hits["flair"] or p
            elif "_seg" in fl or "-seg" in fl or fl.startswith("seg") or "label" in fl or "mask" in fl:
                hits["seg"] = hits["seg"] or p
    return hits

def _z_from_header(path):
    if path is None:
        return np.nan
    return nib.load(path).shape[2]

# ---------- Varredura (2020) ----------
patient_dirs_20 = _list_patient_dirs_20(base_path_20)
assert len(patient_dirs_20) > 0, "Nenhuma pasta BraTS20_Training_* encontrada."

# Acumuladores por sequência
slices_total_by_seq_20 = {m: 0 for m in _modalidades_20}
tumor_slices_by_seq_20 = {m: 0 for m in _modalidades_20}

n_patients_20 = 0

for pf in patient_dirs_20:
    paths = _find_modal_paths_20(pf)
    # paciente conta se tiver ao menos 1 modalidade
    if not any(paths[m] is not None for m in _modalidades_20):
        continue
    n_patients_20 += 1

    # Carrega seg uma única vez (se existir) e vetoriza tumor por fatia
    seg_path = paths["seg"]
    seg_vec = None
    z_seg = 0
    if seg_path is not None:
        seg_vol = nib.load(seg_path).get_fdata()
        z_seg = seg_vol.shape[2]
        seg_vec = np.array([(seg_vol[:, :, z] > 0).any() for z in range(z_seg)], dtype=bool)

    # Para cada sequência, soma Z e tumor slices (limitado por min(Z_seq, Z_seg))
    for m in _modalidades_20:
        p = paths[m]
        if p is None:
            continue
        z_seq = _z_from_header(p)
        if not np.isnan(z_seq):
            slices_total_by_seq_20[m] += int(z_seq)
            if (m != "seg") and (seg_vec is not None):
                # conta quantas fatias do seg têm tumor dentro do intervalo disponível na sequência
                z_use = min(int(z_seq), int(z_seg))
                tumor_slices_by_seq_20[m] += int(seg_vec[:z_use].sum())
            elif m == "seg" and seg_vec is not None:
                tumor_slices_by_seq_20[m] += int(seg_vec.sum())

# ---------- DataFrame (2020) ----------
rows_20 = []
for m in _modalidades_20:
    rows_20.append({
        "dataset": "BraTS2020",
        "sequencia": m,
        "slices_total_seq": slices_total_by_seq_20[m],
        "slices_com_tumor_seq": tumor_slices_by_seq_20[m],
    })

df_2020_summary = pd.DataFrame(rows_20)

# Linha total (soma por sequência; observa que é soma por modalidade, não nº único de fatias)
df_2020_total = pd.DataFrame([{
    "dataset": "BraTS2020",
    "sequencia": "TODAS",
    "slices_totais_dataset": int(df_2020_summary["slices_total_seq"].sum()),
    "slices_totais_com_tumor": int(df_2020_summary["slices_com_tumor_seq"].sum()),
    "pacientes": n_patients_20
}])

# Mostra
df_2020_summary


,dataset,sequencia,slices_total_seq,slices_com_tumor_seq
0,BraTS2020,t1,57040,24354
1,BraTS2020,t1ce,57040,24354
2,BraTS2020,t2,57040,24354
3,BraTS2020,flair,57040,24354
4,BraTS2020,seg,57040,24354


In [6]:
#plot totalizador
df_2020_total

,dataset,sequencia,slices_totais_dataset,slices_totais_com_tumor,pacientes
0,BraTS2020,TODAS,285200,121770,368


In [7]:
# ===== BraTS 2024 LOAD =====
base_path_24 = r"C:\Users\dados\Downloads\BraTS2024-BraTS-GLI-TrainingData\training_data1_v2"

patient_dirs_24 = [
    os.path.join(base_path_24, d)
    for d in os.listdir(base_path_24)
    if os.path.isdir(os.path.join(base_path_24, d)) and d.startswith("BraTS-GLI-")
]
assert len(patient_dirs_24) > 0, "Nenhuma pasta BraTS-GLI-* encontrada."

patient_folder_24 = random.choice(patient_dirs_24)
print("Paciente (2024) escolhido:", os.path.basename(patient_folder_24))

def find_modal_paths_24(patient_folder):
    def patt(suf):
        patterns = [
            os.path.join(patient_folder, f"*_{suf}.nii"),
            os.path.join(patient_folder, f"*_{suf}.nii.gz"),
            os.path.join(patient_folder, f"*-{suf}.nii"),
            os.path.join(patient_folder, f"*-{suf}.nii.gz"),
            os.path.join(patient_folder, f"*{suf}.nii"),
            os.path.join(patient_folder, f"*{suf}.nii.gz"),
        ]
        hits = []
        for p in patterns:
            hits.extend(glob.glob(p))
        return hits
    paths = {
        "t1n": (patt("t1n") or patt("t1native") or [None])[0],
        "t1c": (patt("t1c") or patt("t1ce") or patt("t1gd") or [None])[0],
        "t2w": (patt("t2w") or patt("t2") or [None])[0],
        "t2f": (patt("t2f") or patt("flair") or [None])[0],  # FLAIR equivalente
        "seg": (patt("seg") or patt("label") or patt("mask") or [None])[0],
    }
    miss = [k for k,v in paths.items() if v is None]
    if miss:
        raise FileNotFoundError(f"Arquivos ausentes em {patient_folder}: {miss}")
    return paths

paths24 = find_modal_paths_24(patient_folder_24)

# Carrega volumes (2024)
t1n_24 = nib.load(paths24["t1n"]).get_fdata()
t1c_24 = nib.load(paths24["t1c"]).get_fdata()
t2w_24 = nib.load(paths24["t2w"]).get_fdata()
t2f_24 = nib.load(paths24["t2f"]).get_fdata()   # FLAIR no 2024
seg_24  = nib.load(paths24["seg"]).get_fdata()

mask_bin_24 = (seg_24 > 0).astype(np.uint8)
Z24 = t1n_24.shape[2]

Paciente (2024) escolhido: BraTS-GLI-02787-101


In [8]:
# ===== BraTS 2024 plot =====
def viewer_24(z):
    # Cria a figura
    plt.figure(figsize=(20, 8))

    # ---------- LINHA DE CIMA (imagens brutas + máscara original) ----------

    # (1) T1n
    plt.subplot(2, 5, 1)
    plt.imshow(t1n_24[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T1n")
    plt.axis("off")

    # (2) T1c
    plt.subplot(2, 5, 2)
    plt.imshow(t1c_24[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T1c")
    plt.axis("off")

    # (3) T2w
    plt.subplot(2, 5, 3)
    plt.imshow(t2w_24[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T2w")
    plt.axis("off")

    # (4) T2f / FLAIR
    plt.subplot(2, 5, 4)
    plt.imshow(t2f_24[:, :, z].T, cmap="gray", origin="lower")
    plt.title("T2f / FLAIR")
    plt.axis("off")

    # (5) Máscara ORIGINAL (pode ter múltiplas classes)
    plt.subplot(2, 5, 5)
    plt.imshow(seg_24[:, :, z].T, cmap="nipy_spectral", origin="lower")
    plt.title("Máscara (orig)")
    plt.axis("off")

    # ---------- LINHA DE BAIXO (overlays + máscara binária) ----------

    # Prepara a máscara binária da slice (H, W)
    m = (mask_bin_24[:, :, z].T > 0).astype(np.uint8)

    # (6) T1n + máscara
    plt.subplot(2, 5, 6)
    plt.imshow(t1n_24[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T1n + Másc")
    plt.axis("off")

    # (7) T1c + máscara
    plt.subplot(2, 5, 7)
    plt.imshow(t1c_24[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T1c + Másc")
    plt.axis("off")

    # (8) T2w + máscara
    plt.subplot(2, 5, 8)
    plt.imshow(t2w_24[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T2w + Másc")
    plt.axis("off")

    # (9) T2f / FLAIR + máscara
    plt.subplot(2, 5, 9)
    plt.imshow(t2f_24[:, :, z].T, cmap="gray", origin="lower")
    plt.imshow(m, cmap="Reds", alpha=0.35, origin="lower")
    plt.title("T2f + Másc")
    plt.axis("off")

    # (10) Máscara BINÁRIA
    plt.subplot(2, 5, 10)
    plt.imshow(m, cmap="gray", origin="lower")
    plt.title("Máscara (bin)")
    plt.axis("off")

    # Ajusta o layout
    plt.tight_layout()

    # Exibe
    plt.show()

# Slider
interact(viewer_24, z=(0, Z24-1))


interactive(children=(IntSlider(value=90, description='z', max=181), Output()), _dom_classes=('widget-interact…

<function __main__.viewer_24(z)>

In [9]:
# -------- BraTS 2024 SHAPE --------
assert all(n in globals() for n in ["t1n_24","t1c_24","t2w_24","t2f_24","seg_24"])

rows_2024 = [
    {"dataset":"BraTS2024", "modalidade":"t1n", "H": t1n_24.shape[0], "W": t1n_24.shape[1], "Z": t1n_24.shape[2]},
    {"dataset":"BraTS2024", "modalidade":"t1c", "H": t1c_24.shape[0], "W": t1c_24.shape[1], "Z": t1c_24.shape[2]},
    {"dataset":"BraTS2024", "modalidade":"t2w", "H": t2w_24.shape[0], "W": t2w_24.shape[1], "Z": t2w_24.shape[2]},
    {"dataset":"BraTS2024", "modalidade":"t2f", "H": t2f_24.shape[0], "W": t2f_24.shape[1], "Z": t2f_24.shape[2]},
    {"dataset":"BraTS2024", "modalidade":"seg", "H": seg_24.shape[0],  "W": seg_24.shape[1],  "Z": seg_24.shape[2]},
]
df_2024 = pd.DataFrame(rows_2024).sort_values("modalidade").reset_index(drop=True)
df_2024

,dataset,modalidade,H,W,Z
0,BraTS2024,seg,182,218,182
1,BraTS2024,t1c,182,218,182
2,BraTS2024,t1n,182,218,182
3,BraTS2024,t2f,182,218,182
4,BraTS2024,t2w,182,218,182


In [10]:
# Usa base_path_24 previamente definido no notebook
assert 'base_path_24' in globals(), "Defina base_path_24 antes desta célula."

# ---------- Config (2024) ----------
_modalidades_24 = ["t1n", "t1c", "t2w", "t2f", "seg"]

# ---------- Helpers (2024) ----------
def _list_patient_dirs_24(base, prefix="BraTS-GLI-"):
    return [
        os.path.join(base, d)
        for d in os.listdir(base)
        if os.path.isdir(os.path.join(base, d)) and d.startswith(prefix)
    ]

def _find_modal_paths_24(patient_folder):
    hits = {m: None for m in _modalidades_24}
    for root, _, files in os.walk(patient_folder):
        for f in files:
            fl = f.lower()
            if not (fl.endswith(".nii") or fl.endswith(".nii.gz")):
                continue
            p = os.path.join(root, f)
            if ("t1n" in fl) or ("t1native" in fl):
                hits["t1n"] = hits["t1n"] or p
            elif ("t1c" in fl) or ("t1ce" in fl) or ("t1gd" in fl):
                hits["t1c"] = hits["t1c"] or p
            elif ("t2w" in fl) or (("t2" in fl) and ("flair" not in fl) and ("t2f" not in fl)):
                hits["t2w"] = hits["t2w"] or p
            elif ("t2f" in fl) or ("flair" in fl):
                hits["t2f"] = hits["t2f"] or p
            elif ("_seg" in fl) or (fl.startswith("seg")) or ("-seg" in fl) or ("label" in fl) or ("mask" in fl):
                hits["seg"] = hits["seg"] or p
    return hits

def _z_from_header(path):
    if path is None:
        return np.nan
    return nib.load(path).shape[2]

# ---------- Varredura (2024) ----------
patient_dirs_24 = _list_patient_dirs_24(base_path_24)
assert len(patient_dirs_24) > 0, "Nenhuma pasta BraTS-GLI-* encontrada."

slices_total_by_seq_24 = {m: 0 for m in _modalidades_24}
tumor_slices_by_seq_24 = {m: 0 for m in _modalidades_24}

n_patients_24 = 0

for pf in patient_dirs_24:
    paths = _find_modal_paths_24(pf)
    if not any(paths[m] is not None for m in _modalidades_24):
        continue
    n_patients_24 += 1

    seg_path = paths["seg"]
    seg_vec = None
    z_seg = 0
    if seg_path is not None:
        seg_vol = nib.load(seg_path).get_fdata()
        z_seg = seg_vol.shape[2]
        seg_vec = np.array([(seg_vol[:, :, z] > 0).any() for z in range(z_seg)], dtype=bool)

    for m in _modalidades_24:
        p = paths[m]
        if p is None:
            continue
        z_seq = _z_from_header(p)
        if not np.isnan(z_seq):
            slices_total_by_seq_24[m] += int(z_seq)
            if (m != "seg") and (seg_vec is not None):
                z_use = min(int(z_seq), int(z_seg))
                tumor_slices_by_seq_24[m] += int(seg_vec[:z_use].sum())
            elif m == "seg" and seg_vec is not None:
                tumor_slices_by_seq_24[m] += int(seg_vec.sum())

# ---------- DataFrame (2024) ----------
rows_24 = []
for m in _modalidades_24:
    rows_24.append({
        "dataset": "BraTS2024",
        "sequencia": m,
        "slices_total_seq": slices_total_by_seq_24[m],
        "slices_com_tumor_seq": tumor_slices_by_seq_24[m],
    })

df_2024_summary = pd.DataFrame(rows_24)

df_2024_total = pd.DataFrame([{
    "dataset": "BraTS2024",
    "sequencia": "TODAS",
    "slices_totais_dataset": int(df_2024_summary["slices_total_seq"].sum()),
    "slices_totais_com_tumor": int(df_2024_summary["slices_com_tumor_seq"].sum()),
    "pacientes": n_patients_24
}])

# Mostra
df_2024_summary

,dataset,sequencia,slices_total_seq,slices_com_tumor_seq
0,BraTS2024,t1n,245700,90807
1,BraTS2024,t1c,245700,90807
2,BraTS2024,t2w,245700,90807
3,BraTS2024,t2f,245700,90807
4,BraTS2024,seg,245700,90807


In [11]:
df_2024_total

,dataset,sequencia,slices_totais_dataset,slices_totais_com_tumor,pacientes
0,BraTS2024,TODAS,1228500,454035,1350


In [16]:
# --- Configurar remote e branch (só roda se ainda não tiver feito) ---
!git remote remove origin 2>nul || echo "origin já configurado"
!git remote add origin https://github.com/Henrique-zan/Brain_tumor_segmentation.git
!git branch -M main

# --- Trazer o que existe no GitHub (README, etc.) e mesclar com seus commits locais ---
!git pull origin main --allow-unrelated-histories --rebase=false

# --- Adicionar todos os arquivos (notebooks, .gitignore, etc.) ---
!git add .

# --- Criar commit (se houver alterações) ---
!git commit -m "Base" || echo "nenhuma alteração para commitar"

# --- Enviar para o GitHub ---
!git push -u origin main


Merge made by the 'ort' strategy.
 README.md | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 README.md


From https://github.com/Henrique-zan/Brain_tumor_segmentation
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


On branch main
nothing to commit, working tree clean
"nenhuma alteração para commitar"
branch 'main' set up to track 'origin/main'.


To https://github.com/Henrique-zan/Brain_tumor_segmentation.git
   936f51f..172176b  main -> main
